In [1]:
import os
import random
import torch
from torchvision import transforms
from PIL import Image
import torch.nn as nn
from torchvision import models

In [2]:
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 output classes

model.load_state_dict(torch.load('dfire_resnet50.pth'))
model.eval()

/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/local_scratch/slurm.1310221/ipykernel_1729748/1117793181.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wil

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
images_directory = "../254p RGB Images"
labels_file = "../#10) Frame Pair Labels - modified.txt"

transform = transforms.Compose([
    transforms.Resize((254, 254)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
])

In [4]:
def parse_labels(file_path):
    labels = {}
    with open(file_path, 'r') as f:
        for line in f.readlines()[2:]: 
            line = line.strip()
            if not line: 
                continue
            parts = line.split()
            if len(parts) < 3: 
                continue
            try:
                start, end, label = int(parts[0]), int(parts[1]), parts[2].strip()
                for frame in range(start, end + 1):
                    labels[frame] = 1 if label == 'Y' else 0
            except ValueError:
                continue  
    return labels


In [5]:
labels = parse_labels(labels_file)

image_files = [f for f in os.listdir(images_directory) if f.endswith('.jpg')]

random.seed(42)
test_images = random.sample(image_files, int(len(image_files) * 0.2))

In [6]:
correct = 0
total = 0
for img_name in test_images:
    frame_number = int(img_name.split('(')[1].split(')')[0])  
    label = labels.get(frame_number, None)
    
    if label is None:
        continue 

    img_path = os.path.join(images_directory, img_name)
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0) 

    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    if predicted.item() == label:
        correct += 1
    total += 1

accuracy = correct / total if total > 0 else 0
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 69.16%


In [7]:
import os
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
from PIL import Image

# Lists to store the ground truth labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through the testing dataset
for img_name in test_images:
    frame_number = int(img_name.split('(')[1].split(')')[0])  
    label = labels.get(frame_number, None)
    
    if label is None:
        continue 

    img_path = os.path.join(images_directory, img_name)
    image = Image.open(img_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0) 

    with torch.no_grad():
        output = model(input_tensor)
        _, predicted = torch.max(output, 1)

    # Append true and predicted labels
    true_labels.append(label)
    predicted_labels.append(predicted.item())

# Calculate metrics
precision = precision_score(true_labels, predicted_labels, average='binary')  # 'binary' for 2 classes
recall = recall_score(true_labels, predicted_labels, average='binary')
f1 = f1_score(true_labels, predicted_labels, average='binary')

# Display the metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 0.78
Recall: 0.83
F1 Score: 0.80
